In [133]:
import os
import pandas as pd
import numpy as np
import sys

In [134]:
def GradientStep(X,y,parameters):
    m=X.shape[0]
    y_pred=np.matmul(X,parameters)
    diff=y_pred-y.reshape(-1,1)
    changes=(2/m)*np.matmul(X.T,diff)
    return changes
    

In [135]:
def GradientDescent(X,y,parameters,epochs,learning_rate):
    for i in range(epochs):
        changes=GradientStep(X,y,parameters)
        parameters=parameters-learning_rate*changes
    return parameters

In [136]:
def LinearRegression(X,y,epochs=100,learning_rate=0.0001):
    Ones=np.ones(X.shape[0]).reshape(-1,1)
    X=np.concatenate((Ones,X),axis=1)
    rows=X.shape[0]
    cols=X.shape[1]
    parameters=np.zeros(cols).reshape(-1,1)
    parameters=GradientDescent(X,y,parameters,epochs,learning_rate)
    return parameters

In [137]:
def cost(y_pred,y):
    return np.sum(np.square(y_pred-y))/(len(y))

In [138]:
##Choose Best learning Rate
lr=[ 0.0001*pow(2,i) for i in range(20)]
def best_lr(X,y,lr_list=lr):
    MinCost=sys.maxsize
    BestLR=0
    CostList=[]
    for i in lr_list:
        p=LinearRegression(X,y,learning_rate=i)
        Ones=np.ones(X.shape[0]).reshape(-1,1)
        y_pred=np.matmul(np.concatenate((Ones,X),axis=1),p).T[0]
        temp=cost(y_pred,y.T[0])
        CostList.append((i,temp))
        if(temp<MinCost):
            MinCost=temp
            BestLR=i
    return BestLR


In [139]:
os.chdir("/home/abhishek/Downloads")
data = pd.read_csv("training_ccpp_x_y_train.csv")
data.head()

,# T,V,AP,RH,EP
0,8.58,38.38,1021.03,84.37,482.26
1,21.79,58.20,1017.21,66.74,446.94
2,16.64,48.92,1011.55,78.76,452.56
3,31.38,71.32,1009.17,60.42,433.44
4,9.20,40.03,1017.05,92.46,480.38


In [140]:
X=data.iloc[:,:4].values
y=data.iloc[:,4].values

In [141]:
test_data=pd.read_csv('test_ccpp_x_test.csv',header=None)

In [142]:
X_train=X
y_train=y
X_test=test_data.values
X_test.shape

(2392, 4)

In [143]:
from sklearn.preprocessing import StandardScaler
scX=StandardScaler()
X_train=scX.fit_transform(X_train)
X_test=scX.transform(X_test)
scy=StandardScaler()
y_train=scy.fit_transform(y_train.reshape(-1,1))

In [144]:
def PredictY(X_train,y_train,X_test):
    Lr=best_lr(X_train,y_train.reshape(-1,1))
    parameters=LinearRegression(X_train,y_train.reshape(-1,1),learning_rate=Lr)
    print("Best LR=",Lr)
    Ones=np.ones(X_test.shape[0]).reshape(-1,1)
    X_test=np.concatenate((Ones,X_test),axis=1)
    y_pred=np.matmul(X_test,parameters)
    print("Parameters=",parameters[:,0].tolist())
    return y_pred

In [145]:
y_pred=PredictY(X_train,y_train,X_test)
y_pred=scy.inverse_transform(y_pred)
#y_test=scy.inverse_transform(y_test)
print(y_pred)
#cost(y_pred,y_test)

Best LR= 0.2048
Parameters= [-1.4513132427538757e-15, -0.8632209587661576, -0.17380071075475212, 0.021638534250136652, -0.1345625629717746]
[[469.98098067]
 [471.77625816]
 [433.81828786]
 ...
 [439.16331107]
 [450.77524233]
 [447.315503  ]]


/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  


In [146]:
np.savetxt('ccpp_pred.csv', y_pred[:,0], delimiter=',')